<a href="https://colab.research.google.com/github/IamHeeJ/ML_Ex/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#5-3 트리의 앙상블



**정형 데이터, 비정형 데이터**

- 정형 데이터 : CSV 파일 처럼 구조를 가지고 있는 데이터
- 비정형 데이터 : 텍스트 데이터, 사진, 디지털 음악 등 구조를 가지고있지 않은 데이터 -> 6장

##앙상블 학습(ensemble learning)
정형 데이터를 다루는 데 가장 뛰어난 성과

대다수의 앙상블 학습 알고리즘은 결정 트리를 기반으로 함

##랜덤 포레스트(random forest)
앙상블 학습 중 가장 유명하고 안정적인 성능 제공

여러개의 결정 트리를 랜덤하게 만들고 각 결정 트리의 예측을 사용해 최종 예측 출력
1. 훈련하기 위한 데이터를 랜덤하게 만듬 (중복 가능)
2. 부트스트랩 샘플로 결정 트리를 훈련
3. 정해진 수 만큼 위 과정을 반복

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


이 예제는 매우 간단하고 특성도 적어서 그리드 서치를 사용하더라도 하이퍼 튜닝의 결과가 크게 나아지지 않음

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


**OOB특성**

부트스트랩 샘플에 포함되지 않는 샘플

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


##엑스트라 트리

랜덤 포레스트와 비슷하게 동작
  - 부트스트랩 샘플을 사용하지 않고 각 결정 트리

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜던 포레스트와 비슷한 결과를 얻음
보통 엑스트라 트리가 무작위성이 더 크기 때문에 더 많은 하..

In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


##Bagging vs Boosting
위 두 방법은 배깅(bagging) 방법임

여러 개의 모델을 랜덤하게 만들고 각 모델의 결과를 종합하여 결과를 출력

아래 방법들은 부스팅(boosting) 방식임

모델을 순차적으로 여러개 만들되 이전 모델을 보완한 새로운 모델을 만들고 최종적으로 생성된 모든 모델을 하나로 합침

##그레디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식

과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

그레디언트란 이름이 붙은 이유는 경사 하강법을 사용하여 앙상블에 추가하기 때문
  - 분류에서는 로지스틱 손실 함수 사용
  - 회귀에선 평균 제곱 오차 함수 사용

학습률 매개변수로 속도 조절

예측에 사용될 데이터가 많은 경우 사용

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


##히스토그램 기반 그레디언트 부스팅

- 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기 많음
- 입력 특성을 256구간으로 나누고 하나를 뗴어놓고 누락된 값을 위해 사용
- 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 누락된 특성이 있더라도 이를 따로 전처리할 필요 x

- 히스토그램 : 데이터를 일정한 구간으로 쪼개서 막대 그래프로 표현

In [ ]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대 적합 억제하면서 그레디언트 부스팅보다 높은 성능을 제공

- permutation_importances()
  - 특성을 하나씩 랜덤하게 섞어서 모델의 성능 변화를 관찰하여 어떤 특성이 중요한지 계산

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

###XGBoost
사이킷런의 cross_validate()함수 사용 가능

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

###LightGBM


In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))